# Deploying a Named Entity Recognition Model in Riva

[Train Adapt Optimize (TAO) Toolkit](https://developer.nvidia.com/tao-toolkit)  provides the capability to export your model in a format that can deployed using [NVIDIA Riva](https://developer.nvidia.com/riva), a highly performant application framework for multi-modal conversational AI services using GPUs. 

This tutorial explores taking a .riva model, the result of `tao token_classification` command, and leveraging the Riva ServiceMaker framework to aggregate all the necessary artifacts for Riva deployment to a target environment. Once the model is deployed in Riva, you can issue inference requests to the server. We will demonstrate how quick and straightforward this whole process is. 

---
## Learning Objectives
In this notebook, you will learn how to:  
- Use Riva ServiceMaker to take a TAO exported .riva and convert it to .rmir
- Deploy the model(s) locally  on the Riva Server
- Send inference requests from a demo client using Riva API bindings.

---
## Pre-requisites

To follow along, please make sure:
- You have access to NVIDIA NGC, and are able to download the Riva Quickstart [resources](https://ngc.nvidia.com/catalog/resources/nvidia:riva:riva_quickstart)
- Have a .riva model file that you wish to deploy. You can obtain this from `tao <task> export` (with `export_format=RIVA`). Please refer the tutorial on *Named entity recognition using Train Adapt Optimize (TAO) Toolkit* for more details on training and exporting a .riva model.

---
## Riva ServiceMaker
Servicemaker is the set of tools that aggregates all the necessary artifacts (models, files, configurations, and user settings) for Riva deployment to a target environment. It has two main components as shown below:

### 1. Riva-build

This step helps build a Riva-ready version of the model. It’s only output is an intermediate format (called a RMIR) of an end to end pipeline for the supported services within Riva. We are taking a NER BERT Model in consideration<br>

`riva-build` is responsible for the combination of one or more exported models (.riva files) into a single file containing an intermediate format called Riva Model Intermediate Representation (.rmir). This file contains a deployment-agnostic specification of the whole end-to-end pipeline along with all the assets required for the final deployment and inference. Please checkout the [documentation](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/service-nlp.html#pipeline-configuration) to find out more.

In [ ]:
# IMPORTANT: UPDATE THESE PATHS 

# ServiceMaker Docker
RIVA_SM_CONTAINER = "<add container name>"

# Directory where the .riva model is stored $MODEL_LOC/*.riva
MODEL_LOC = "<add path to model location>"

# Name of the .riva file
MODEL_NAME = "<add model name>"

# Key that model is encrypted with, while exporting with TAO
KEY = "<add encryption key used for trained model>"

In [ ]:
# Get the ServiceMaker docker
!docker pull $RIVA_SM_CONTAINER

In [ ]:
# Syntax: riva-build <task-name> output-dir-for-rmir/model.rmir:key dir-for-riva/model.riva:key
!docker run --rm --gpus 0 -v $MODEL_LOC:/data $RIVA_SM_CONTAINER -- \
        riva-build token_classification /data/token-classification.rmir:$KEY /data/$MODEL_NAME:$KEY

### 2. Riva-deploy

The deployment tool takes as input one or more Riva Model Intermediate Representation (RMIR) files and a target model repository directory. It creates an ensemble configuration specifying the pipeline for the execution and finally writes all those assets to the output model repository directory.

In [ ]:
# Syntax: riva-deploy -f dir-for-rmir/model.rmir:key output-dir-for-repository
! docker run --rm --gpus 0 -v $MODEL_LOC:/data $RIVA_SM_CONTAINER -- \
            riva-deploy -f  /data/token-classification.rmir:$KEY /data/models/

---
## Start Riva Server
Once the model repository is generated, we are ready to start the Riva server. From this step onwards you need to download the Riva QuickStart Resource from NGC. 
Set the path to the directory here:

In [ ]:
# Set the Riva QuickStart directory
RIVA_DIR = "<Path to the uncompressed folder downloaded from quickstart(include the folder name)>"

Next, we modify config.sh to enable relevant Riva services (nlp for token classification), provide the encryption key, and path to the model repository (`riva_model_loc`) generated in the previous step among other configurations. 

For instance, if above the model repository is generated at `$MODEL_LOC/models`, then you can specify `riva_model_loc` as the same directory as `MODEL_LOC` <br>

Pretrained versions of models specified in models_asr/nlp/tts are fetched from NGC. Since we are using our custom model, we can comment it in models_asr (and any others that are not relevant to your use case). <br>

`NOTE:` Please perform the step of editing config.sh outside this notebook.

#### config.sh snipet
```
# Enable or Disable Riva Services 
service_enabled_asr=false                                 ## MAKE CHANGES HERE
service_enabled_nlp=true                                  ## MAKE CHANGES HERE
service_enabled_tts=false                                 ## MAKE CHANGES HERE

# Specify one or more GPUs to use
# specifying more than one GPU is currently an experimental feature, and may result in undefined behaviours.
gpus_to_use="device=0"

# Specify the encryption key to use to deploy models
MODEL_DEPLOY_KEY="tlt_encode"                             ## Set the model encryption key

# Locations to use for storing models artifacts
...
riva_model_loc="<add path>"                              ## Replace with MODEL_LOC

# The default RMIRs are downloaded from NGC by default in the above $riva_rmir_loc directory
# If you'd like to skip the download from NGC and use the existing RMIRs in the $riva_rmir_loc
# then set the below $use_existing_rmirs flag to true.
...
use_existing_rmirs=true                                  ## Set to True
```

In [ ]:
# Ensure you have permission to execute these scripts
! cd $RIVA_DIR && chmod +x ./riva_init.sh && chmod +x ./riva_start.sh

In [ ]:
# Run Riva Init. This will fetch the containers/models
# YOU CAN SKIP THIS STEP IF YOU DID RIVA DEPLOY
! cd $RIVA_DIR && ./riva_init.sh config.sh

In [ ]:
# Run Riva Start. This will deploy your model(s).
! cd $RIVA_DIR && ./riva_start.sh config.sh

---
## Run Inference
Once the Riva server is up and running with your models, you can send inference requests querying the server. 

To send GRPC requests, you can install Riva Python API bindings for client. This is available as a pip .whl with the QuickStart.

Otherwise, you can use the riva-client docker container which comes pre-installed with all the inference dependancies.

In [ ]:
# Install client API bindings
! cd $RIVA_DIR && pip install <add .whl file>

### Connect to Riva server and run inference
Now we actually query the Riva server, let's get started. The following cell queries the riva server(using grpc) to yield a result.

In [ ]:
%%writefile $RIVA_DIR/ner_client.py

import grpc
import os
import argparse
import riva_api.riva_nlp_pb2 as rnlp
import riva_api.riva_nlp_pb2_grpc as rnlp_srv

# use the NER network to return top-1 classes for entities
def postprocess_labels_server(tokens_response):
    results = []
    for i in range(0, len(tokens_response.results)):
        slots = []
        slot_scores = []
        tokens = []
        for j in range(0, len(tokens_response.results[i].results)):
          entity = tokens_response.results[i].results[j]
          tokens.append(entity.token)
          slots.append(entity.label[0].class_name)
          slot_scores.append(entity.label[0].score)
        results.append((slots, tokens, slot_scores))

    return results

def run_ner(grpc_server, query):
    channel = grpc.insecure_channel(grpc_server)
    riva_nlp = rnlp_srv.RivaLanguageUnderstandingStub(channel)
    req = rnlp.AnalyzeEntitiesRequest()
    req.query = query
    resp = riva_nlp.AnalyzeEntities(req)
    print("Query:", query)
    print(postprocess_labels_server(resp))

def get_args():
    parser = argparse.ArgumentParser(description="Client app to test named entity recognition on Riva")
    parser.add_argument("--server", default="localhost:50051", type=str, help="URI to GRPC server endpoint")
    parser.add_argument("--query", default="NVIDIA is located at Santa Clara", type=str, help="Input Query")
    return parser.parse_args()

def run_ner_client():
    args = get_args()
    run_ner(args.server, query=args.query)

if __name__ == '__main__':
    run_ner_client()

If you've installed the Riva Python API bindings for client using the pip whl, you can execute the above saved script with the following command:

In [ ]:
#! python3 $RIVA_DIR/ner_client.py --query "NVIDIA is located at Santa Clara" --server localhost:50051

Otherwise, to use the Riva client docker container, you can execute the below docker run command with the right `RIVA_CLIENT_CONTAINER` name.

In [ ]:
RIVA_CLIENT_CONTAINER = "<add container name>"

In [ ]:
!docker run -d --privileged \
    -v $RIVA_DIR:/riva \
    --net=host --rm \
    --name riva-client \
    $RIVA_CLIENT_CONTAINER sleep 1h

Finally, executing the client python script in the above client container

In [ ]:
!docker exec riva-client python3 /riva/ner_client.py --query "NVIDIA is located at Santa Clara" --server localhost:50051

Make sure to close the riva-client docker container before shutting down the jupyter kernel. In addition, do close the riva services container with the `riva_stop.sh` script in the quickstart

In [ ]:
! docker stop riva-client
! cd $RIVA_DIR && ./riva_stop.sh config.sh